In [1]:
'''%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here
%pip install openml
%pip install numpy
%pip install pandas
# cf. https://pytorch.org/get-started/locally/
#%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # cpu version
%pip install -U scikit-learn
%pip install scipy
%pip install -U pymoo
%pip list'''

'%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here\n%pip install openml\n%pip install numpy\n%pip install pandas\n# cf. https://pytorch.org/get-started/locally/\n#%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command\n%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # cpu version\n%pip install -U scikit-learn\n%pip install scipy\n%pip install -U pymoo\n%pip list'

In [2]:
from openml import tasks

from classes import EAGGA

import os

In [3]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)

oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openml/tasks/functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openml/tasks/functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


In [4]:
hps = {
    'total_layers': (3, 10),
    'nodes_per_hidden_layer': (3, 20),
    'mu': 100,
    'lambda': 10,
    'holdout_train_size': 2/3,
    'cv_k': 5
}

batch_size = 64
min_epochs = 200
patience = 100

secs_per_fold = 60
secs_total = 8 * 60 * 60

pareto_fronts = list()
for (oml_dataset, class_positive) in zip(oml_datasets, positive_classes):
    name = oml_dataset.name
    print(f'Dataset {name}')

    file_path = os.path.join('export', name)
    
    eagga = EAGGA(
        oml_dataset=oml_dataset,
        class_positive=class_positive,
        hps=hps,
        batch_size=batch_size,
        min_epochs=min_epochs,
        patience=patience,
        secs_per_fold=secs_per_fold,
        secs_total=secs_total,
        file_path=file_path
    )
    #eagga.load_population(3)
    
    pareto_front = eagga.run_eagga()
    pareto_fronts.append(pareto_front)
pareto_fronts

Dataset diabetes
Starting init population
Finished init population
Start EAGGA at 2025-03-03T18:42:28.668796
Generation 1, evaluate 5 individuals
Running 5-fold CV for individual 1/5: 3 total layers, 3 nodes per hidden layer, dropout p 0.2, gs: ([0, 2, 3, 4, 5, 6, 7], [[[1], 1]])
Fold 1/5 | trained for 199 epochs / 5.001 seconds | stopped early: False | metrics: {'loss': 0.6577245593070984, 'auc': 0.7541459369817579, 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Fold 2/5 | trained for 175 epochs / 4.482 seconds | stopped early: True | metrics: {'loss': 0.6044129729270935, 'auc': 0.8517827529021559, 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Fold 3/5 | trained for 179 epochs / 5.064 seconds | stopped early: False | metrics: {'loss': 0.6576378345489502, 'auc': 0.7121535181236675, 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Fold 4/5 | trained for 190 epochs / 4.957 seconds | stopped early: True | metrics: {'loss': 0.6299660503864288, 'auc': 0.8057659932659933, 'nf': 0.125, 'ni': 0.0, 'nnm': 0.0}
Fold 5/5 | traine

KeyboardInterrupt: 